In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import simplejson as json
import ipywidgets as widgets
from ipywidgets import Output, VBox
from plotly.tools import FigureFactory as ff

In [2]:
plotly.offline.init_notebook_mode(connected=True)

In [3]:
def make_scatter(startX, startY, successX, successY, stoppedX, stoppedY, motorColor):
        start = go.Scatter(
            name = 'Event Initiated',
            mode = 'markers',
            marker = dict(
            color = str(motorColor),
            size = 15,
            ),
            hoverinfo = 'x+y',
            x=startX,
            y=startY
        )
        stop = go.Scatter(
               name = 'Event Completed',
               mode = 'markers',
               marker = dict(
               color = 'black',
               size = 8
               ),
               hoverinfo = 'none',
               x=successX,
               y=successY
           )
        stopped = go.Scatter(
               name = 'Event Stopped',
                mode = 'markers',
                marker = dict(
                color = 'red',
                size = 15
                ),
                hoverinfo = 'y',
                x=stoppedX,
                y=stoppedY
            )
        if stoppedX!=None:
            data = [start, stop, stopped]
        else:
            data = [start, stop]
        return data
    
def plot_motor(motor_name, time1, time2):
        colors = 2.5*(ord(motor_name[:1])),2.5*ord(motor_name[1:2]),2.5*(ord(motor_name[2:3]))
        motorColor = 'rgb'+str(colors)
        with open('../data/transfocator.txt', 'r') as handle:
            df = pd.read_json(handle)
            timeList = []
            positionList = []
            stoppedX = []
            stoppedY=[]        
            startX = []
            startY = []
            successX = []
            successY = []
            stopEvent = None
            for i in range(0, df.shape[0]):
                timeList.append(datetime.fromtimestamp(df['start_ts'][i]))
                timeList.append(datetime.fromtimestamp(df['finish_ts'][i]))
                positionList.append(df['start_pos'][i])
                positionList.append(df['finish_pos'][i])
                startX.append(datetime.fromtimestamp(df['start_ts'][i]))
                startY.append((df['start_pos'][i]))
                if df['success'][i]==False:
                    m = (df['finish_pos'][i]-df['start_pos'][i])/(df['finish_ts'][i]-df['start_ts'][i])
                    targetTime = df['target'][i]/m-df['finish_pos'][i]/m+df['finish_ts'][i]
                    
                    stoppedX.append(datetime.fromtimestamp(targetTime))
                    if motor_name=='MFX':
                        stoppedY.append(df['target'][i])
                    else:
                        stoppedY.append(df['target'][i]-4)
                    stopEvent = go.Scatter(
                        x=(datetime.fromtimestamp(df['finish_ts'][i]), datetime.fromtimestamp(targetTime)),
                        y=(df['finish_pos'][i], df['target'][i]),
                        showlegend = False,
                        hoverinfo = 'none',
                        text = 'Hypothetical Complete Event',
                        line = dict(
                        color='red'
                        )
                    )
                else:
                    successX.append(datetime.fromtimestamp(df['finish_ts'][i]))
                    successY.append(df['finish_pos'][i])
            event = go.Scatter(
                              x=timeList,
                              y=positionList,
                              hoverinfo = 'none',
                              showlegend = False,
                              line = dict(
                              color=motorColor
                              )
            )
            if stoppedX!=[]:
                data = [event]+[stopEvent]+make_scatter(startX, startY, successX, successY, stoppedX, stoppedY, motorColor)
            else:
                data =  [event]+make_scatter(startX, startY, successX, successY, stoppedX, stoppedY, motorColor)
            for i in range(0, df.shape[0]):
                df.loc[i, 'start_ts'] = str(datetime.fromtimestamp(df['start_ts'][i]))[:-7]
                df.loc[i, 'finish_ts'] = str(datetime.fromtimestamp(df['finish_ts'][i]))[:-7]
                df.loc[i, 'finish_pos'] = round(df['finish_pos'][i])
                df.loc[i, 'start_pos'] = round(df['start_pos'][i])
            return [data, df]
            
                
                
                
                

In [4]:
layout = go.Layout(showlegend = True,
    title=go.layout.Title(
        text='Motor Status',
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Time of Event',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Motor Position',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )   
                    )
motorList = ('AMO', 'CXI', 'MEC', 'MFX', 'SXR', 'XCS', 'XPP')
motors = widgets.SelectMultiple(options = motorList, value = ('MFX',), description='Instrument(s)', disabled=False)
start_time = widgets.DatePicker(description='Start Date', disabled=False)
stop_time = widgets.DatePicker(description='End Date', disabled=False)

def add_motors(motors, time1, time2):
    data=[]
    tables = []
    for m in motors:
        colors = (ord(m[:1]))*2.5,2.5*ord(m[1:2]),2.5*(ord(m[2:3]))
        motorColor = 'rgb'+str(colors)
        data = data+plot_motor(m, time1, time2)[0]
        tabledat = plot_motor(m, time1, time2)[1]
        table = go.Figure(
        [go.Table(header={'values': tabledat.columns, 'fill' : dict(color=motorColor)}, 
                  cells={'values': [tabledat[i] for i in tabledat.columns]})],
        )  
        tables.append(table)
    f = go.Figure(data=data, layout=layout)
    fig = go.FigureWidget(f)
    scatter = fig.data[0]
    display(fig)
    for i in range(0, len(tables)):
        plotly.offline.iplot(tables[i])
    def make_table(trace, points, state):
        print(points.xs)
        #plotly.offline.iplot(table)
    scatter.on_click(make_table)
widgets.interactive(add_motors, motors= motors, time1=start_time, time2=stop_time)

interactive(children=(SelectMultiple(description='Instrument(s)', index=(3,), options=('AMO', 'CXI', 'MEC', 'M…